Authors: Jana & Rebecka 

# Woman and War Rhetoric in American Congress Speeches 
 

> ### Data:  
**hein-daily.zip** and **hein-bound.zip**from https://data.stanford.edu/congress_text#download-data Congressional Record for the 43rd-114th Congresses , the daily edition covers 97th to 114th with a time span from 1981 up to 2017, the bound edition from 1873 up to 2011


#### Our hypothesis is that there is a noticible difference between topics in speech of women and men during the cold war area in congress speeches.

The cold war period was spanning from 12. März 1947 up to 26. Dezember 1991 during that time many events happened that changed the world like for example the proxy wars that broke out. 

---

In our analysis we will proceed the following way:

> 1.) Preprocessing: Extracting data




C:\Users\rebec\Anaconda3\envs\rebeckafahrni\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (0,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,0,1,2
283232,970283394,Without objection. it is so ordered.,NaN
283233,970283395,Mr. President. now I am happy to report on beh...,NaN
283234,970283396,Mr. President. no less a constitutional expert...,NaN
283235,970283397,which commended President -Reagans START propo...,NaN
283236,970283398,MWr. President. if there is no further busines...,NaN


the ***SpeakerMap***-data is structured in the following way:

'#nr_SpeakerMap.txt' (#nr. ranging from 097-114 (hein))

>speakterid|speech_id|lastname|firstname|chamber|state|gender|party|district|nonvoting

---

In [4]:
import re
import pandas
import fnmatch
import os 

files = fnmatch.filter(os.listdir(r'C:\Users\rebec\Desktop\hein-daily'), '*_SpeakerMap.txt')
#print(files) all files of speakersmap
l = []
for file in files:
    
    df1 = pd.read_csv(file,sep='|')
    l.append(df1)
speakersmap = pd.concat(l, axis=0, ignore_index=True)
speakersmap.tail(3)

FileNotFoundError: [Errno 2] No such file or directory: '097_SpeakerMap.txt'

In [5]:
#print(speakersmap.isnull().sum()) # nans' in district -> as we are also interested in the district we should remove all rows containing -nans in district
speakers_df = speakersmap.dropna(axis=0)
speakers_df.head()

NameError: name 'speakersmap' is not defined

The ***descr***-data is structured in the following way:

'descr_#nr.'  #nr. ranging from 097-114 (hein)

>speech_id|chamber|date|number_within_file|speaker|first_name|last_name|state|gender|line_start|line_end|file|char_count|word_count

In [6]:
import re
import pandas
import fnmatch
import os 

files = fnmatch.filter(os.listdir(r'C:\Users\rebec\Desktop\hein-daily'), 'descr_*.txt')
#print(files) all files of descr
l = []
for file in files:
    df1 = pd.read_csv(file,sep='|',na_values='Unknown')
    l.append(df1)
decr_df = pd.concat(l, axis=0, ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'descr_097.txt'

In [7]:
# still contains 'Unknowns' in first_name/last_name or state see decr_df.head()
decr_df.head()

# total missing values for each column
print(decr_df.isnull().sum())   # most missing data comes from first_names followed by state and gender

# I decide to remove the rows with contain 'unknown'-data
decr_df = decr_df.dropna(axis=0)
decr_df.head()

NameError: name 'decr_df' is not defined

The ***byspeaker_2gram***-data is structured in the following way:

'byspeaker_2gram_#nr.'  #nr. ranging from 097-114 (hein)

> speakerid|phrase|count

In [8]:
import re
import pandas
import fnmatch
import os 

files = fnmatch.filter(os.listdir(r'C:\Users\rebec\Desktop\hein-daily'), 'byspeaker_2gram_*.txt')
#print(files) all files of speakersmap
l = []
for file in files:
    df1 = pd.read_csv(file,sep='|')
    l.append(df1)
byspeaker2gram_df = pd.concat(l, axis=0, ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'byspeaker_2gram_097.txt'

In [9]:
byspeaker2gram_df.head(10)
print(byspeaker2gram_df.isnull().sum()) # no nans

NameError: name 'byspeaker2gram_df' is not defined

The ***byparty_2gram***-data is structured in the following way:

'byparty_2gram_#nr.'
> party|phrase|count

In [10]:
import re
import pandas as pd
import fnmatch
import os 

files = fnmatch.filter(os.listdir(r'C:\Users\rebec\Desktop\hein-daily'), 'byparty_2gram_*.txt')
#print(files) all files of speakersmap
l = []
for file in files:
    df1 = pd.read_csv(file,sep='|')
    l.append(df1)
byparty_df = pd.concat(l, axis=0, ignore_index=True)
byparty_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'byparty_2gram_097.txt'

In [ ]:
byparty_df.head()
print(bbyparty_df.isnull().sum())

The ***speeches***-data is structured in the following way:

'speeches_#nr.'
>speech_id|speech

In [11]:
import pandas
import numpy as np

# create nice dataframe -> speechid | speech

# in order to anaylise the speeches
# preprocessing is necessairy

# getting rid of non-important speeeches for ecample thankyou or other very short phrases

# filter out '|' in order to make the speech-ends
# the topic -> BERTopic can be used, need to get rid of stopwords
import fnmatch
# concatinate all Speeches form 097-114th Congress -> create dataframe 

files = fnmatch.filter(os.listdir(r'C:\Users\rebec\Desktop\hein-daily'), 'speeches_*')
print(files)
li=[]

for i,file in enumerate(files):
    with open(file, 'r',encoding='iso-8859-1') as temp_f:
        # get No of columns in each line
        col_count = [ len(l.split("|")) for l in temp_f.readlines() ]

        # Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
        column_names = [i for i in range(0, max(col_count))]
        #print(column_names)
        # Read txt
        df = pd.read_csv(file, delimiter="|", names=column_names)
        li.append(df)
        #print(df.head(1))
    
frame = pd.concat(li, axis=0, ignore_index=True)
frame.drop(frame.columns[len(frame.columns)-41], axis=1, inplace=True) # delete unnecessary
df_speeches = frame.rename(columns=frame.iloc[0]).drop(frame.index[0]) # replace header

['speeches_097.txt', 'speeches_098.txt', 'speeches_099.txt', 'speeches_100.txt', 'speeches_101.txt', 'speeches_102.txt', 'speeches_103.txt', 'speeches_104.txt', 'speeches_105.txt', 'speeches_106.txt', 'speeches_107.txt', 'speeches_108.txt', 'speeches_109.txt', 'speeches_110.txt', 'speeches_111.txt', 'speeches_112.txt', 'speeches_113.txt', 'speeches_114.txt']


FileNotFoundError: [Errno 2] No such file or directory: 'speeches_097.txt'

In [ ]:
df_speeches.head()